This code is intended to take a list of weblinks and search for all the text within.
It returns a database of gtd_id,link,text


In [1]:
'''dependencies'''
import pandas as pd

from bs4 import BeautifulSoup
from collections import Counter
import re
from urllib.parse import urlparse
import time

import requests


In [2]:
#df = pd.read_csv('dl_df.csv').drop(['Unnamed: 0','city','year','country','datetime','search_terms'],1)
#df

In [3]:
df = pd.read_csv('0_2015-2017_db_links.csv').drop('Unnamed: 0',1)
df

,gtd_id,link,search_terms,country
0,201712310032,http://www.cnn.com/2010/TECH/01/14/twitter.hoa...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
1,201712310032,https://www.npr.org/2017/10/29/560569373/are-s...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
2,201712310032,https://www.quora.com/What-is-the-difference-b...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
3,201712310032,http://www.courtisane.be/sites/default/files/l...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
4,201712310032,http://www.protectingeducation.org/sites/defau...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
5,201712310032,http://www.tandfonline.com/doi/pdf/10.1080/045...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
6,201712310032,http://eresources.nlb.gov.sg/newspapers/digiti...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
7,201712310032,http://eresources.nlb.gov.sg/newspapers/digiti...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
8,201712310032,http://www.forestpeoples.org/sites/fpp/files/p...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines
9,201712310032,https://adst.org/wp-content/uploads/2018/04/Ph...,Cotabato City OR Maguindanao OR Philippines OR...,Philippines


In [5]:
df2 = pd.DataFrame()
sample_df = df
sample_links = [i for i in sample_df['link']]
len(sample_links)


17536

In [5]:
%%time
#conda install tornado==4.5.3 as tornado 5x is not good for this

import asyncio
import aiohttp
import ssl


url_list = sample_links #just increase numbers

async def fetch(session, url):
    async with session.get(url, ssl=ssl.SSLContext()) as response:
        return await response.text()
    
async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results
    
def p_soup(page):
    try:
        soup = BeautifulSoup(page, 'lxml')
        tags = soup.find_all('p')
        tags_list = [t.text for t in tags]
        return tags_list
    except:
        tags_list = []
        return tags_list


Wall time: 2.28 s


In [15]:
%%time
'''this is for weird cases where code does not work for mass '''
total = 5007
a = 4500
b = 4510
while b <= 5000:
    loop = asyncio.new_event_loop()
    print('Getting ' +str(a) + ' to ' +str(b))
    urls = url_list[a:b]
    htmls = loop.run_until_complete(fetch_all(urls, loop))
    plist = list(map(p_soup,htmls))
    lst = list(zip(url_list,plist))
    l_df = pd.DataFrame(lst)
    #df2 = pd.concat([df2,l_df])
    a += 10
    b += 10
    l_df.to_csv('2015_2017_3.csv',mode='a',header= False)
    loop.close()

Getting 4500 to 4510
Getting 4510 to 4520
Getting 4520 to 4530
Getting 4530 to 4540
Getting 4540 to 4550
Getting 4550 to 4560
Getting 4560 to 4570
Getting 4570 to 4580
Getting 4580 to 4590
Getting 4590 to 4600
Getting 4600 to 4610
Getting 4610 to 4620
Getting 4620 to 4630
Getting 4630 to 4640
Getting 4640 to 4650
Getting 4650 to 4660
Getting 4660 to 4670
Getting 4670 to 4680
Getting 4680 to 4690
Getting 4690 to 4700
Getting 4700 to 4710
Getting 4710 to 4720
Getting 4720 to 4730


Fatal error on SSL transport
protocol: <asyncio.sslproto.SSLProtocol object at 0x00000110C0BEB710>
transport: <_SelectorSocketTransport closing fd=2352 read=idle write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "C:\Users\Xingyu\Anaconda3\lib\asyncio\sslproto.py", line 638, in _process_write_backlog
    ssldata, offset = self._sslpipe.feed_appdata(data, offset)
AttributeError: 'NoneType' object has no attribute 'feed_appdata'
Fatal error on SSL transport
protocol: <asyncio.sslproto.SSLProtocol object at 0x00000110C0BEB710>
transport: <_SelectorSocketTransport closing fd=2352 read=idle write=<idle, bufsize=0>>
Traceback (most recent call last):
  File "C:\Users\Xingyu\Anaconda3\lib\asyncio\sslproto.py", line 638, in _process_write_backlog
    ssldata, offset = self._sslpipe.feed_appdata(data, offset)
AttributeError: 'NoneType' object has no attribute 'feed_appdata'


Getting 4730 to 4740
Getting 4740 to 4750
Getting 4750 to 4760
Getting 4760 to 4770
Getting 4770 to 4780
Getting 4780 to 4790
Getting 4790 to 4800
Getting 4800 to 4810
Getting 4810 to 4820
Getting 4820 to 4830
Getting 4830 to 4840
Getting 4840 to 4850
Getting 4850 to 4860
Getting 4860 to 4870
Getting 4870 to 4880
Getting 4880 to 4890
Getting 4890 to 4900
Getting 4900 to 4910
Getting 4910 to 4920
Getting 4920 to 4930
Getting 4930 to 4940
Getting 4940 to 4950
Getting 4950 to 4960
Getting 4960 to 4970
Getting 4970 to 4980
Getting 4980 to 4990
Getting 4990 to 5000
Wall time: 59min 12s


In [2]:
import tornado
print(tornado.version)

4.5.3
